In [1]:
#import libraries

import scipy.sparse as sps
import numpy as np
import os
import pandas as pd 

URM_all= sps.load_npz("./Processed Matrices/URM_simple_coo_2020.npz").tocsr()
ICM_all= sps.load_npz("./Processed Matrices/ICM_simple_coo_2020.npz").tocsr()

In [2]:
%load_ext Cython

In [3]:
from Notebooks_utils.evaluation_function import evaluate_algorithm
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

np.random.seed(41148)
URM_train, URM_valid = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [4]:
from GraphBased.RP3betaRecommender import RP3betaRecommender
RP3brecommender= RP3betaRecommender(URM_train)
RP3brecommender.fit(topK=770, alpha=0.414615342821205, beta=0.17823717015919388, normalize_similarity=False)

RP3betaRecommender: URM Detected 82 (1.03 %) cold users.
RP3betaRecommender: URM Detected 2372 (9.13 %) cold items.


In [5]:
from MatrixFactorization.IALSRecommender import IALSRecommender
IALSrecommender= IALSRecommender(URM_train)
IALSrecommender.fit(alpha = 49.99999999999999, epochs=10, num_factors=200, confidence_scaling="linear",epsilon=0.002936450234353087, reg=0.01)

IALSRecommender: URM Detected 82 (1.03 %) cold users.
IALSRecommender: URM Detected 2372 (9.13 %) cold items.
IALSRecommender: Epoch 1 of 10. Elapsed time 16.65 sec
IALSRecommender: Epoch 2 of 10. Elapsed time 36.30 sec
IALSRecommender: Epoch 3 of 10. Elapsed time 56.02 sec
IALSRecommender: Epoch 4 of 10. Elapsed time 1.26 min
IALSRecommender: Epoch 5 of 10. Elapsed time 1.61 min
IALSRecommender: Epoch 6 of 10. Elapsed time 1.96 min
IALSRecommender: Epoch 7 of 10. Elapsed time 2.31 min
IALSRecommender: Epoch 8 of 10. Elapsed time 2.66 min
IALSRecommender: Epoch 9 of 10. Elapsed time 3.01 min
IALSRecommender: Epoch 10 of 10. Elapsed time 3.35 min
IALSRecommender: Terminating at epoch 10. Elapsed time 3.35 min


In [6]:
from KNN.ItemKNNScoresHybridRecommender import ItemKNNScoresHybridRecommender

recommender=ItemKNNScoresHybridRecommender(URM_train, RP3brecommender, IALSrecommender)
evaluator_validation = EvaluatorHoldout(URM_valid, cutoff_list=[10])
resultDF= pd.DataFrame({"alpha":[],"MAP":[]})

for alpha in range(0,101,2):
    recommender.fit(alpha/100)
    MAP,_=evaluator_validation.evaluateRecommender(recommender)
    MAP=MAP[10]["MAP"]
    resultDF=resultDF.append({"alpha":[alpha/100],"MAP":[MAP]},ignore_index=True)
    print(alpha/100, MAP)

ItemKNNScoresHybridRecommender: URM Detected 82 (1.03 %) cold users.
ItemKNNScoresHybridRecommender: URM Detected 2372 (9.13 %) cold items.
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 5.78 sec. Users per second: 979
0.0 0.05765672866741284
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 5.00 sec. Users per second: 1130
0.02 0.057942161310212424
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 5.02 sec. Users per second: 1128
0.04 0.05853611826394703
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 5.00 sec. Users per second: 1131
0.06 0.059253429987476264
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 4.99 sec. Users per second: 1134
0.08 0.059575879085094284
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 5.02 sec. Users per second: 1128
0.1 0.06013170907950977
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 5.03 sec. Users per second: 1124
0.12 0.060452147611150996
EvaluatorHoldout: Processed 5657 ( 100.00% ) in 4.99 sec. Users per second: 1133
0.14 0.06065879698274142
EvaluatorHo

In [7]:
resultDF_sorted=resultDF.sort_values(by=['MAP'],ascending=False)
resultDF_sorted.to_csv("Hybrid_IALS_RP3b_results_sorted.csv")

In [8]:
RP3brecommender_full= RP3betaRecommender(URM_all)
RP3brecommender_full.fit(topK=770, alpha=0.414615342821205, beta=0.17823717015919388, normalize_similarity=False)

IALSrecommender_full= IALSRecommender(URM_all)
IALSrecommender_full.fit(alpha = 49.99999999999999, epochs=10, num_factors=200, confidence_scaling="linear",epsilon=0.002936450234353087, reg=0.01)

recommender_full= ItemKNNScoresHybridRecommender(URM_all, RP3brecommender_full, IALSrecommender_full)
recommender_full.fit(alpha=resultDF_sorted.iloc[0]['alpha'][0])

RP3betaRecommender: URM Detected 1079 (4.15 %) cold items.
IALSRecommender: URM Detected 1079 (4.15 %) cold items.
IALSRecommender: Epoch 1 of 10. Elapsed time 20.88 sec
IALSRecommender: Epoch 2 of 10. Elapsed time 41.93 sec
IALSRecommender: Epoch 3 of 10. Elapsed time 1.05 min
IALSRecommender: Epoch 4 of 10. Elapsed time 1.41 min
IALSRecommender: Epoch 5 of 10. Elapsed time 1.77 min
IALSRecommender: Epoch 6 of 10. Elapsed time 2.12 min
IALSRecommender: Epoch 7 of 10. Elapsed time 2.47 min
IALSRecommender: Epoch 8 of 10. Elapsed time 2.81 min
IALSRecommender: Epoch 9 of 10. Elapsed time 3.14 min
IALSRecommender: Epoch 10 of 10. Elapsed time 3.49 min
IALSRecommender: Terminating at epoch 10. Elapsed time 3.49 min
ItemKNNScoresHybridRecommender: URM Detected 1079 (4.15 %) cold items.


In [9]:
rec_df=pd.DataFrame({"user_id":[],"item_list":[]})
target_user_list=list(pd.read_csv("./Data/2020/data_target_users_test.csv").user_id)

for uid in target_user_list:
    rec_list=map(str,recommender_full.recommend(uid)[:10])
    rec_df=rec_df.append({"user_id":uid,"item_list":' '.join(rec_list)},ignore_index=True)

    
rec_df=rec_df.astype({"user_id":'int32',"item_list":'str'})
rec_df=rec_df.set_index('user_id')
rec_df.to_csv("Hybrid_IALS_RP3b_recommendations.csv")